   # Laboratoire 1 ele8702 
   ## Hiver 2024

## Introduction

Pour faire suite au Lab0 dans lequel vous avez appris à trouver les coordonnées des Antennes et des UEs, vous allez travailler dans ce laboratoire sur le processus d'association des UEs aux antennes utilisant la métrique de pertes en espace libre (pathloss) donnée par les formules de Okumura-Hata. Dans le prochain laboratoire nous utiliserons par contre les formules du 3GPP (voir [ici](https://www.etsi.org/deliver/etsi_tr/138900_138999/138901/16.01.00_60/tr_138901v160100p.pdf) sections 7.4.1 et 7.4.2).




## Énoncé du problème

Nous avons un terrain rectangulaire. Nous devons placer sur ce terrain **25 antennes** et **300 UEs**. Toutes les antennes qui seront placées sur le terrain sont du même type. Par contre, les UEs peuvent être de type différent. Chaque type de UE est caractérisé par l'application qu'il contient. Parmi les 300 UEs, **100 vont tourner l'application 1 et 200 l'application 2.** Les antennes doivent être placées **sur une grille** alors que les UEs doivent être placés de **facon aléatoire.**  Nous avons une base de données (fichier ```devices_db.yaml```) qui spécifie les caractéristiques des différents types d'équipements (antennes et UEs). Cette base de données vous sera fournie (voir section d'élements fournis). 

## Objectifs
- Simuler l'assignation des UEs aux antennes dans un réseau cellulaire
    - Considérer **la perte en espace libre minimale** entre un UE et toutes les antennes comme métrique d'association en utilisant les formules de Okumura-Hata 
    
       

## Données d'entrée

  - **fichier de cas**:
       - créé pour chaque cas en étude. Vous devez modifier le fichier de cas utilisé dans le Lab0 pour considérer les modèles de pertes tel qu'expliqué ci-dessous.    
        
  - **base de donnée des équipements (fichier ```devices_db.yaml```)**:
       - elle est unique pour tous les cas d'étude. Elle a été légèrement modifiée par rapport au Lab0 (voir ci-dessous). 

### Modification de format d'un fichier de cas

- Le fichier de cas sera modifié pour tenir compte des spécificités de la formule de calcul de pertes en espace libre (pathloss).
- La modification du fichier de cas consiste à éliminer la section SCENARIO du Lab0 et ajouter la section PATHLOSS. Le PATHLOSS est caractérisé par le modèle de pathloss (i.e. 3gpp, okumura, COST, etc) et par le nom spécifique du scénario de chaque modèle (i.e., le ```UMI```, ```RMa```, etc pour le 3gpp, ```urban_large```, ```urban_small```, ```suburban``` ou``` open``` pour Okumura-Hata )

```python
###############################################
#                                             #
#         Case 5:                             #
#            File: case5.yaml                 #
#            Description: Etude de pathloss   #
#                                             #
###############################################
ETUDE_PATHLOSS:
   PATHLOSS :
       model : 
       scenario : 
```

## Élements Fournis

### 1. Base de données des dispositifs ```devices_db.yaml``` 

Son utilisation est obligatoire. Pour le lab1 elle ne peut pas être modifiée. De nouveaux modèles de dispositifs ont été ajoutés par rapport au Lab0.

```python
################################################
#                                              #
#     devices_db:                              #
#     File: devices_db.yaml                    #
#     Description: Base de données avec        #
#                  info des dispositifs        #
#                                              #
################################################
ANTENNAS:
  Antenna1:
    type: antenna
    name: Antenna1
    height: 10 
    frequency:  28 #GHz
    gain:  40
#
  Antenna2:
    type: antenna
    name: Antenna2
    height: 35
    frequency:  0.9 #GHz
    gain:  45
#
  Antenna3:
    type: antenna
    name: Antenna3
    height: 10
    frequency:  28 #GHz
    gain:  45
#
  Antenna4:
    type: antenna
    name: Antenna4
    height: 37
    frequency:  0.9 #GHz
    gain:  45
        
#
   Antenna5:
    type: antenna
    name: Antenna5
    height: 37
    frequency:  0.250 #GHz
    gain:  30 
        
#
   Antenna6:
    type: antenna
    name: Antenna6
    height: 20
    frequency:  1.4 #GHz
    gain:  30 
        
#
   Antenna7:
    type: antenna
    name: Antenna7
    height: 30
    frequency:  3.2 #GHz
    gain:  35 
        
UES:
  UE1-App1:
    type: ue
    name: UE1-App1
    app: app1
    height:  1.5

  UE2-App2:
    type: ue
    name: UE2-App2
    app: app2
    height:  1.5
```

### 2. Classes

Les classes ci-dessous montrent les attributs obligatoires qui décrivent les objects Antenna et UE. D'autres attributs et ou méthodes peuvent être ajoutés.

In [11]:
class Antenna:

     def __init__(self, id):
        self.id = id          #id de l'antenne (int)
        self.frequency = None # Antenna frequency in GHz
        self.height = None    # Antenna height
        self.group = None     # group défini dans la base de données (str)
        self.coords = None    # tuple contenant les coordonnées (x,y) 
        self.assoc_ues = []   # liste avec les id des UEs associés à l'antenne
        self.scenario = None  # pathloss scénario tel que lu du fichier de cas (str)
        self.gen = None       # type de géneration de coordonnées: 'g', 'a', etc. (str)

    
    
class UE:

     def __init__(self, id, app_name):
        self.id= id           # id de l'UE (int)
        self.height = None    # UE height
        self.group = None     # group défini dans la base de données (str)
        self.coords=None      # tuple contenant les coordonnées (x,y) 
        self.app=app_name     # nom de l'application qui tourne dans le UE (str)
        self.assoc_ant=None   # id de l'antenne associée à l'UE (int)
        self.los = True       # LoS ou non (bool)
        self.gen = None       # type de géneration de coordonnées: 'g', 'a', etc. (str)
    

    

### 3. Fonction d'erreur

In [12]:
def ERROR(msg , code = 1):
    print("\n\n\nERROR\nPROGRAM STOPPED!!!\n")
    if msg:
        print(msg)
    print(f"\n\texit code = {code}\n\n\t\n")
    sys.exit(code)

## Composantes indispensables du programme ```lab1_eqn.py```

- Le programme doit respecter la structure suivante (la même utilisée dans le lab0). En plus des fonctions que vous allez créér, il est impératif que les fonctions ci-dessous existent. La façon de démarrer votre programme reste la même que pour le lab0, c-à-d que que la fonction ```main ()``` est reponsable du démarrage du restant du programme. 

- La fonction ```lab0 (data_case)``` du laboratoire précédent est maintenant appelée ```lab1 (data_case)```. 

        .
        .
        .
        .
     Reste du programme (Vos fonctions)
        .
        .
        .
        .

In [ ]:
def lab1 (data_case):
    #TODO ....
    # antennas est une liste qui contient les objets de type Antenna
    # ues est une liste qui contient les objets de type UE
    #
    # antennas = [ant0,ant1,...] 
    #            ant1, ant2 etc sont des instances (objets) de la classe Antenna
    # ues = [ue0, ue1,...] 
    #             ue0, ue1, etc sont des instances (objets) de la classe UE
    # avant de faire le retour, les objets appartenant aux listes antennas et ues 
    # doivent avoir leur coordonées initialisées
    # CETTE FONCTION EST OBLIGATOIRE
    return (antennas,ues)
    
def okumura (argument1, argument2, ...):
    #TODO ....
    # C'est à vous décider le nombre et type d'arguments utilisés pour
    #faire le calcul de pathloss avec le modèle d'Okumura-Hata
    # CETTE FONCTION EST OBLIGATOIRE
    return pathloss

def treat_cli_args(arg):
    # arg est une liste qui contient les arguments utilisés lors de l'appel du programme par CLI. 
    # Cette fonction doit retourner le nom du fichier de cas à partir de l'interface de commande (CLI)
    #... 
    # TODO
    #....
    # CETTE FONCTION EST OBLIGATOIRE
    # À noter que dans cette fonction il faut ajouter les vérifications qui s'imposent
    # par exemple, nombre d'arguments appropriés, existance du fichier de cas, etc.
    return case_file_name

def main(arg):
    case_file_name = treat_cli_args(arg)
    data_case = read_yaml_file(case_file_name)
    #
    #TODO les instructions de main qui vont faire appel aux autres fonctions du programme
    #.....

if __name__ == '__main__':
    # sys.argv est une liste qui contient les arguments utilisés lors de l'appel 
    # du programme à partir du CLI. Cette liste est créée automatiquement par Python. Vous devez 
    # juste inscrire l'argument tel que montré ci-dessous.
    main(sys.argv[1:])



   - Les classes Antenna et UE ci-dessus doivent apparaître tel quel dans votre programme (fournies)
   - La fonction ```get_lattice_coords``` vous permet de trouver des coordonnées en grille (fournie lab0)
   - La fonction ```get_from_dict``` vous permet de sortir la valeur d'une clé d'un dictionnaire (fournie lab0)
   - La fonction ```read_yaml_file``` vous permet de lire un fichier ```.yaml``` (fournie lab0)    - La fonction ```gen_random_coords``` doit trouver des coordonnées aléatoires (développée lab0)
   - La fonction ```lab1``` doit retourner une liste des objets antenna et une liste des objets ues (à développer)
   - La fonction ```treat_cli_args``` retourne le nom du fichier de cas à partir de l'interface de commande (CLI) développée au lab0)
   - La fonction ```main``` qui contient des instructions pour faire appel aux autres fonctions du programme (à développer). 
   - La fonction ```okumura``` doit faire le calcul de pathloss entre un UE et une antenna en utilisant les formules de Okumura-Hata pour un scenario donné. Elle doit retourner une valeur de pathloss.
   - **Note: les définitions de fonctions main et treat_cli_args ainsi que l'appel de main doivent OBLIGATOIREMENT respecter la syntaxe montrée ci-haut**

    
       définition de main:
             ```def main(arg):``` <br>
     
       appel de main:
            ``` main(sys.argv[1:])``` <br>
          
       définition de treat_cli_args:
            ``` treat_cli_args(arg):```

## Livrables


 1a) Le programme python, ```lab1_eqn.py```. 
Ce programme doit obligatoirement démarrer, **sans erreurs**, à partir du terminal (CLI) avec l'instruction ```python lab1_eqn.py lab1_eqn_cas.yaml```

 1b) Tout autre fichier necéssaire à faire rouler le programme décrit dans 1a).
   
   **Note**: ne pas oublier de substituer le ```n``` par le numéro de votre équipe.   

2. Un **fichier de pathloss**. Ce fichier de texte doit être appelé ```lab1_eqn_pl.txt```. Il contient les valeurs de pathloss calculées par le modèle Okumura-Hata avec le scénario de votre choix (```urban_large```, ```urban_small```, ```suburban``` ou``` open```). Ce fichier doit contenir le pathloss de **toutes les combinaisons** UE-Antennes. Le format de chaque ligne du fichier doit être le suivant:

    - première colonne: id de l'UE
    - deuxième colonne: id de l'antenne
    - troisième colonne: pathloss entre l'UE et l'antenne
    - quatrième colonne: modèle de pathloss utilisé (```okumura```) écrit tout en minuscule
    - cinquième colonne : scénario considéré 
    
    Les colonnes doivent obligatoirement être séparées exclusivement par des espaces (un ou plus). Ne pas ajouter aucune autre information ou ligne supplémentaire. 
    
    Voici un exemple: 

In [ ]:
                   .
                   .
                   .
0	21	136.23413791338908	okumura	urban_large
0	22	140.53336631359366	okumura	urban_large
0	23	144.1946635361489	okumura	urban_large
0	24	147.26220692687684	okumura	urban_large
1	0	147.37758299886258	okumura	urban_large
1	1	145.1453323273901	okumura	urban_large
1	2	143.06428369505159	okumura	urban_large
1	3	141.55139566627122	okumura	urban_large
1	4	141.11628892983626	okumura	urban_large
                   .
                   .
                   .

Dans l'exemple ci-haut le pathloss entre le UE 0 et l'antenne 24 est de 147.26... dB.
Le pathloss entre le UE 1 et l'antenne 3 est de 141.55... dB.

3. Un fichier **d'association d'antennes**. Ce fichier de texte doit être appelé ```lab1_eqn_assoc_ant.txt```. Il montre **pour chaque antenne**, la liste des ids des UEs qu'y sont associés dans le cas des pathloss calculés. Utiliser le format suivant:

    - première colonne: id de l'antenne
    - deuxième colonne, troisième colonne, etc. les id des UEs associés à l'antenne.
      Si aucun UEs n'y est rattaché, laisser un blanc après avoir marqué le id de l'antenne
      
Les ids des UEs associés à l'antenne devront se trouver dans l'attribut ```assoc_ues``` de chaque objet de type Antenna

Exemple de fichier d'association d'antennes

In [ ]:
0	3	6
1	14	29	42
2
3	7	15	45	46
4	19	25	28	34
5	22
6
7
.
.
.

Dans cet exemple l'antenne 3 est associée aux UEs 7,15, 45 et 46. Par contre, les antennes 2, 6 et 7 n'ont aucun UE associé.

4.  Un fichier **d'association d'UEs**. Ce fichier de texte doit s'appeler ```lab1_eqn_assoc_ue.txt```. Il montre **pour chaque UE** l'antenne à laquelle il est associé. Le format doit être le suivant:
    - première colonne: id de l'UE
    - deuxième colonne : id de l'antenne associée
    
Le id de l'antenne associée à l'UE doit se trouver dans l'attribut ```assoc_ant``` de chaque objet de type UE.

In [ ]:
Exemple de fichier d'association d'UEs

In [ ]:
0	10
1	24
2	20
3	0
4	22
5	20
.
.
.

Cet exemple montre que le UE 2 est associé à l'antenne 20 et que le UE 3 est associé à l'antenne 0.

5. Le fichier de coordonnées appelé ```lab1_eqn_coords.txt``` dont le format a été donné au lab0. 

**Note:** contrairement au lab0, imprimer toutes les chiffres significatifs (pas seulement un chiffre décimal).

6. Le fichier de cas utilisé appelé ```lab1_eqn_cas.yaml``` ainsi que le fichier ```devices_db.yaml```

## Réquis et autres détails

- Le programme doit tourner, dans tout ordinateur, sans aucune manipulation ni ajout de fichiers ou de commandes. **L'utilisateur ne devra jamais avoir à manipuler de quelque façon que ce soit le code source**.
- Tout changement de valeurs doit être communiqué à partir du fichier de cas.
- Le programme doit fonctionner avec **n'importe lequel fichier de cas** (respectant le format indiqué) fourni par l'utilisateur.
- Tous les ```Réquis pour tous les laboratoires``` indiqués au lab0 s'appliquent.
- Les fichiers doivent être déposés individuellement sur Moodle, sans faire des zip et sans avoir des structures hiérarchiques.
- Un seul dépôt par équipe.


## Barême de correction

1. Structure du programme (25%)
			
2. Robustesse du programme (30%)	
	

3. Résultats (30%)
		
4. Suivi des consignes (15%)
	
**Note** Si le programme ne demarre pas quand on utilise la commande CLI indiquée , il y a une pénalité minimale de 40%




